#### CSCE 489 :: Recommender Systems :: Texas A&M University :: Spring 2021


# Homework 2: Collaborative Filtering

### 100 points [10% of your final grade]

- **Due Friday, February 26 by 11:59pm**

*Goals of this homework:* The objective of this homework is to turn the theory of collaborative filtering into practice, and compare the results with your naive non-personalized recommendation from Homework 1 to see how personalized collaborative filtering algorithms provide better recommendations for both the rating prediction task (with explicit feedback) and the ranking task (with implicit feedback).

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, if your UIN is `123456789`, then your homework submission would be `123456789_hw2.ipynb`. Submit this notebook via Canvas. Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit. 

*Late policy:* You may use up to three of your late days on this assignment. No homeworks will be accepted after March 1 11:59pm.

## Collaboration Declaration:

***You must add all of your collaboration declarations here. Who did you talk to about this assignment? What web resources did you use? Etc.***

For example:
* Part 1a: I talked to Amy about how to split the data randomly. She helped me understand that I needed to use a random number generator.
* Part 1b: I needed help on how to comment my code, so I relied on this StackOverflow thread: https://stackoverflow.com/questions/184618/what-is-the-best-comment-in-source-code-you-have-ever-encountered
* (Replace this bullet list with your own collaboration declarations.)


# Part 1. Recommendations with User Ratings (Explicit Feedback) (60 points total)

In this first part, similar to Part 1 in our Homework 1, we still focus on the rating prediction recommendation task with explicit feedback. But in this part, you will need to build **personalized** models instead of non-personalized models as in Homework 1. You will also evaluate your personalized models to compare them with the non-personalized one in Homework 1.

For this part, we will:

* load and process the MovieLens 1M dataset, 
* build a baseline estimation model,
* build a user-user collaborative filtering model,
* improve the user-user collaborative filtering model, and
* evaluate and compare these different models.

Before you start to build your recommendation models. First, we need to load and preprocess the experiment dataset. We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this homework, and use the same loading and preprocessing process as Homework 1 Part 1a. The code has been provided in the next cell, and you need to run it. The resulting data variables are the same as those in Homework 1: train_mat is the numpy array variable for training data of size (#users, #items) with non-zero entries representing user-item ratings, and zero entries representing unknown user-item ratings; and test_mat is the numpy array variable for testing data of size (#users, #items).

In [17]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])

# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

<ipython-input-17-e069bfd8a143>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])


## Part 1a: Build the Baseline Estimation Model (20 points)

First, let's implement a simple personalized recommendation model -- the baseline estimate -- introduced in class: $b_{u,i}=\mu+b_i+b_u$, where $\mu$ is the overall mean rating for all items, $b_u$ = average rating of user $u-\mu$, $b_i$ = average rating of item $i-\mu$. Store your prediction as a numpy array variable 'prediction_mat' of size (#users, #movies) with each entry showing the predicted rating for the corresponding user-movie pair.

* Hint: for users who do not have ratings in train_mat, set $b_u=0$ for them; and for movies which do not have ratings in train_mat, set $b_i=0$ for them

In [18]:
# calculate the prediction_mat by the baseline estimation recommendation algorithm
# Your Code Here...

indicator_mat = (train_mat > 0).astype(float)
mu = np.sum(train_mat) / np.sum(indicator_mat)

num_rating_items = np.sum(indicator_mat, axis=0, keepdims=True)
num_rating_items[num_rating_items == 0] = 1
mu_items = np.sum(train_mat, axis=0, keepdims=True) / num_rating_items
mu_items[mu_items == 0] = mu

num_rating_users = np.sum(indicator_mat, axis=1, keepdims=True)
num_rating_users[num_rating_users == 0] = 1
mu_users = np.sum(train_mat, axis=1, keepdims=True) / num_rating_users
mu_users[mu_users == 0] = mu
bi = mu_items - mu
bu = mu_users - mu
prediction_mat = np.ones_like(train_mat)
prediction_mat = mu + bi + bu

Now, with this prediction_mat based on the baseline estimate, let's use the same RMSE metric as you used in Homework 1 Part 1c to evaluate the quality of the baseline estimate model. Please print out the RMSE of your prediction_mat using test_mat in the next cell.

In [16]:
# calculate and print out the RMSE for your prediction_df and the test_df
# Your Code Here...

indicator_mat = (test_mat > 0).astype(float)
test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

0.0


Do you observe an increase or decrease of the RMSE compared with the result you got from Homework 1 Part 1c with a non-personalized recommendation model? What do you think leads to this difference?

*Write your answer here*

Yes. Because the baseline estimation consider both user and item baselines togather but non-personalized recommendation model only considers item average rating. 

## Part 1b: User-user Collaborative Filtering with Jaccard Similarity (30 points)

In this part, you need to build a user-user collaborative filtering recommendation model with **Jaccard similarity** to predict user-movie ratings. 

The prediction of the score for a user-item pair $(u,i)$ should use the formulation: $p_{u,i}=\bar{r}_u+\frac{\sum_{u^\prime\in N}s(u,u^\prime)(r_{u^\prime,i}-\bar{r}_{u^\prime})}{\sum_{u^\prime\in N}|s(u, u^\prime)|}$ as introduced in class, where $s(u, u^\prime)$ is the Jaccard similarity. We set the size of $N$ as 10.

In the next cell, you need to write your code to implement this algorithm, and generate a numpy array variable named 'prediction_mat' of size (#user, #movie) with each entry showing the predicted rating for the corresponding user-movie pair.

* Hint: when you find the nearest neighbor set $N$ of a user $u$, do not include user $u$ in $N$  


In [5]:
# calculate the prediction_mat by your user-user collaborative filtering recommendation algorithm
# Your Code Here...

# binary matrix to indicate whether there is a rating for a user-movie pair
indicator_mat = (train_mat > 0).astype(float)  # size = (#user, #movie)  

# calculate the number of ratings for each user
num_rating_per_user = np.sum(indicator_mat, axis=1, keepdims=True)  # size = (#user, 1)  

# calculate the numerator of Jaccard similarity: for two users, calculate the number of movies both of they rated
numerator = np.matmul(indicator_mat, indicator_mat.T)  # size = (#user, #user)

# calculate the denominator of Jaccard similarity: for two users, calculate the number of movies they rated in total
denominator = num_rating_per_user + num_rating_per_user.T - numerator  # size = (#user, #user)

# set 0 to be 1 to avoid error in division 
denominator[denominator == 0] = 1

# calculate Jaccard similarity matrix
Jaccard_mat = numerator / denominator  # size = (#user, #user)

prediction_mat = train_mat.copy()

num_rating_users[num_rating_users == 0] = 1
mu_users = np.sum(train_mat, axis=1, keepdims=True) / num_rating_users
deviation_mat = (train_mat - mu_users) * indicator_mat
for u in range(num_user):
    similarities = Jaccard_mat[u, :]
    similarities[u] = -1
    N_idx = np.argpartition(similarities, -10)[-10:]
    N_sim = similarities[N_idx]
    prediction_mat[u, :] = np.sum(N_sim.reshape((-1, 1)) * deviation_mat[N_idx, :], axis=0) / np.sum(N_sim)
prediction_mat += mu_users

Please print out the RMSE of your prediction_mat using test_mat in the next cell.

In [6]:
# calculate and print out the RMSE for your prediction_df and the test_df
# Your Code Here...

indicator_mat = (test_mat > 0).astype(float)
test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

0.9824264022914787


Comparing the RMSE results of this user-user collaborative filtering, the baseline estimate algorithm, and your non-personalized recommendation model from Homework 1 Part 1c, what do you observe? Is the user-user collaborative filtering the one producing the best performance? What reasons do you think can explain what you observe?

*Write your answer here...*

No, user-user CF does not perform as well as the baseline estimation model. A possible reason may be in this data, the user-user similarity is not informative enough for accurate recommendation.

## Part 1c: Improve the Collaborative Filtering Model (20 points)

Next, you need to try your best to improve the collaborative filtering model so that we can improve our RMSE! This is open-ended, so feel free to try whatever collaborative filtering tricks you like. We talked about several in class, plus you can find more in the readings. As a starting point, here are some possibilities:

* Try the cosine similarity or pearson similarity measurement instead of Jaccard
* Try item-item collaborative filtering instead of user-user CF
* Try to include the baseline estimation model in your collaborative filtering model

Besides these, you can also figure out your own solutions how to improve the performance. Write your code in the next cell and print out the RMSE of your new model.

In [14]:
# implement your improved model and print out the RMSE
# Your Code Here...

indicator_mat = (train_mat > 0).astype(float)
num_rating_items = np.sum(indicator_mat, axis=0, keepdims=True)
numer = np.matmul(indicator_mat.T, indicator_mat)  # num_item * num_item
denom = num_rating_items.T + num_rating_items - numer  # num_item * num_item
denom[denom == 0] = 1
J = numer / denom  # num_item * num_item

prediction_mat = train_mat.copy()

num_rating_items[num_rating_items == 0] = 1
mu_items = np.sum(train_mat, axis=0, keepdims=True) / num_rating_items  # 1 * num_item
deviation_mat = (train_mat - mu_items) * indicator_mat
for i in range(num_movie):
    similarities = J[i, :]
    similarities[i] = -1
    N_idx = np.argpartition(similarities, -10)[-10:]
    N_sim = similarities[N_idx]
    prediction_mat[:, i] = np.sum(N_sim.reshape((1, -1)) * deviation_mat[:, N_idx], axis=1) / (np.sum(N_sim) + 1e-10)
prediction_mat += mu_items

indicator_mat = (test_mat > 0).astype(float)

test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

0.013411736923351136


And please briefly explain what your new model does to improve the performance.

*Briefly explain what you do in your new model to improve the performance.
Write your answer here...*

Use item-item CF.

# Part 2. Recommendations with implicit feedback (40 points total)

Now we turn to study how collaborative filtering algorithms work for recommendations with implicit feedback. The overall pipeline is the same as in Part 2 of Homework 1. But now you need to implement a user-user collaborative filtering algorithm for recommendation.

In this part, you will use the same MovieLens 1M dataset, and:
* write the code to implement a user-user collaborative filtering algorithm,
* and evaluate your recommender.

Before the algorithm implementation, we need to first transfer the explicit datasets you already generated to implicit ones. The process is the same as Part 2a in Homework 1, and the code has been provided. You need to run the code in the next cell.

In [8]:
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

Then, you need to write code to implement a user-user collaborative filtering algorithm with **cosine similarity**. Although we have not discussed in class how to use collaborative filtering for implicit feedback, it is quite straightforward. 

* We first need to calculate the cosine similarity between users based on the binary feedback vectors of users; 
* Then, for a specific user $u$, we predict a preference vector for the user $u$ by weighted averaging the binary feedback vectors of N users who are most similar to $u$;
* And last, rank the movies based on the predicted preference vector of the user $u$ as recommendations. 

The predicted preference score from user $u$ to movie $i$ can be calculated as: $p_{u,i}=\frac{\sum_{u^\prime\in N}s(u,u^\prime)r_{u^\prime,i}}{\sum_{u^\prime\in N}|s(u,u^\prime)|}$, where $s(u,u^\prime)$ is the cosine similarity, and we set the size of $N$ as 10.

In the next cell, write your code to generate the ranked lists of movies by this user-user collaborative filtering recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. 

* Hint: for a user $u$, the movies user $u$ liked in the train_mat should be excluded in the top 50 recommendation list

In [9]:
# Your Code Here...

user_train_like = []
for u in range(num_user):
    user_train_like.append(np.where(train_mat[u, :] > 0)[0])

numer = np.matmul(train_mat, train_mat.T)
denom = np.sum(train_mat ** 2, axis=1, keepdims=True) ** 0.5
Cosine = numer / np.matmul(denom, denom.T)

recommendation = []
for u in range(num_user):
    similarities = Cosine[u, :]
    similarities[u] = -1
    N_idx = np.argpartition(similarities, -10)[-10:]
    N_sim = similarities[N_idx]
    scores = np.sum(N_sim.reshape((-1, 1)) * train_mat[N_idx, :], axis=0) / np.sum(N_sim)
    
    train_like = user_train_like[u]
    scores[train_like] = -9999
    top50_iid = np.argpartition(scores, -50)[-50:]
    top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
    recommendation.append(top50_iid)
recommendation = np.array(recommendation)

Last, you need to evaluate your user-user collaborative filtering algorithm by the held-out testing dataset test_mat for each user. Here, we use the same metrics recall@k and precision@k we used in Part 2c of Homework 1. So you can use the same code here to calculate recall@k and precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

In [10]:
# Calculate recall@k, precision@k with k=5, 20, 50 and print out the average over all users for these 6 metrics.
# Your Code Here...

user_test_like = []
for u in range(num_user):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])
    
recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

for u in range(num_user):
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))

recall@5	[0.070814],	||	 recall@20	[0.191537],	||	 recall@50	[0.319397]
precision@5	[0.380430],	||	 precision@20	[0.291598],	||	 precision@50	[0.221063]
